In [1]:
import os
import pandas as pd
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
model = models.resnet18(pretrained=True)
model = torch.nn.Sequential(*list(model.children())[:-1])
model.eval()

In [3]:
def cos_sim(im1, im2):
    return np.dot(im1, im2) / (np.linalg.norm(im1) * np.linalg.norm(im2))

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

def get_features(model, im_path, transform):
    im = Image.open(im_path).convert('RGB')
    im = im.resize((256,256))
    img_tensor = transform(im)
    features_tensor = model(img_tensor.unsqueeze(0))
    vector = features_tensor.cpu().detach().numpy()
    flat = np.ndarray.flatten(vector)
    return flat



In [5]:

def calcul_simi(model, path_img, path_dir):
    target_features = get_features(model, path_img, transform)

    image_files = [img for img in os.listdir(path_dir) if os.path.isfile(os.path.join(path_dir, img)) and img.lower().endswith(('.jpeg', '.jpg', '.png'))]

    similarities = []

    for image_file in image_files:
        image_path = os.path.join(path_dir, image_file)
        if image_path != path_img:
            features = get_features(model, image_path, transform)
            similarity = cos_sim(target_features, features)
            similarities.append((similarity, image_path))

    similarities.sort(reverse=True, key=lambda x: x[0])
    sim_top5 = similarities[:5]

    fig, axes = plt.subplots(1, 6, figsize=(20, 10))

    target_img = Image.open(path_img)
    axes[0].imshow(target_img)
    axes[0].set_title('Objet de référence')
    axes[0].axis('off')

    for i, (sim, img_path) in enumerate(sim_top5):
        img = Image.open(img_path)
        axes[i + 1].imshow(img)
        axes[i + 1].set_title(f'{i + 1}')
        axes[i + 1].axis('off')

    plt.show()
    return sim_top5


In [24]:
path_dir = './mini_corpus'
path_img = ''

In [ ]:
with torch.no_grad():
    sim_top5=calcul_simi(model, path_img, path_dir)
